Split_data

In [ ]:
char_to_num = layers.StringLookup(
    vocabulary=list(characters), mask_token=None
)

num_to_char = layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

def split_data(images, labels, train_size=0.8, shuffle=True):
    size = len(images)
    np.random.seed(42)
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    train_samples = int(size * train_size)
    val_test_size = (size - train_samples) // 2
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:train_samples+val_test_size]], labels[indices[train_samples:train_samples+val_test_size]]
    x_test, y_test = images[indices[train_samples+val_test_size:]], labels[indices[train_samples+val_test_size:]]
    return x_train, x_valid, x_test, y_train, y_valid, y_test

x_train, x_valid, x_test, y_train, y_valid, y_test = split_data(np.array(path_list), np.array(labels_list))

Prepare Data

In [ ]:
import tensorflow as tf
import numpy as np
char_to_num = layers.StringLookup(
    vocabulary=list(characters), mask_token=None
)

num_to_char = layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

def split_data(images, labels, train_size=0.8, shuffle=True):
    size = len(images)
    np.random.seed(42)
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    train_samples = int(size * train_size)
    val_test_size = (size - train_samples) // 2
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:train_samples+val_test_size]], labels[indices[train_samples:train_samples+val_test_size]]
    x_test, y_test = images[indices[train_samples+val_test_size:]], labels[indices[train_samples+val_test_size:]]
    return x_train, x_valid, x_test, y_train, y_valid, y_test

x_train, x_valid, x_test, y_train, y_valid, y_test = split_data(np.array(path_list), np.array(labels_list))
batch_size = 16
img_height = 50
padding_token = 11

def augment_image_targeted(img, label):
    label = tf.strings.to_number(label, out_type=tf.int32)

    if tf.equal(label, 2):
        img = tf.image.random_contrast(img, lower=0.7, upper=1.3)
        img = tf.image.random_brightness(img, max_delta=0.2)
    elif tf.equal(label, 5):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_contrast(img, lower=0.7, upper=1.3)
    elif tf.equal(label, 3):
        img = tf.image.random_flip_up_down(img)
    elif tf.equal(label, 4):
        img = tf.image.random_flip_left_right(img)
    elif tf.equal(label, 0):
        img = tf.image.random_brightness(img, max_delta=0.2)
    elif tf.equal(label, 7):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_contrast(img, lower=0.7, upper=1.3)
    elif tf.equal(label, 8):
        angles = tf.random.uniform([], -0.2, 0.2)
        radians = angles * np.pi
        img = tf.image.rot90(img, k=tf.cast(tf.random.uniform([], 0, 4), tf.int32))
        img = tf.image.random_brightness(img, max_delta=0.2)
    elif tf.equal(label, 1):
        img = tf.image.random_flip_left_right(img)
    elif tf.equal(label, 6):
        img = tf.image.random_flip_up_down(img)
    elif tf.equal(label, 9):
        img = tf.image.random_flip_left_right(img)

    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, lower=0.7, upper=1.3)
    return img, label

def encode_single_sample(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, tf.shape(img)[1]])
    img = tf.transpose(img, perm=[1, 0, 2])

    label = tf.strings.as_string(label)
    label = tf.strings.unicode_split(label, input_encoding="UTF-8")
    label = char_to_num(label)

    length = tf.shape(label)[0]
    pad_amount = max_length - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)

    return img, label

padded_shapes = ([None, None, 1], [None])
padding_values = (0.0, np.int64(0))

def map_fn(image, label):
    return {'image': image, 'label': label}

train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .map(augment_image_targeted, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size, padded_shapes=padded_shapes, padding_values=padding_values)
    .map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
    .cache()
)

validation_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
validation_dataset = (
    validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size, padded_shapes=padded_shapes, padding_values=padding_values)
    .map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
    .cache()
)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = (
    test_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size, padded_shapes=padded_shapes, padding_values=padding_values)
    .map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
